Semana 9 - Reto Calidad del agua

Lázaro Lara Martínez. Matricula A01793198
José Mtanous Treviño. Matricula A00169781

Ciencia y Analítica de datos.

Profesor Titular. Jobish Vallikavungal Devassia
Profesor Tutor. Mtro. Mario Alberto Solano Saldaña

09/Noviembre\/2022

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [23]:
# cargamos nuestro dataset y lo inspeccionamos rapídamente
path_escalas_subterranea = '../Reto/datos/Escalas_subterranea.csv'
path_calidad_subterranea = '../Reto/datos/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv'

dfEscalasSubterranea = pd.read_csv(path_escalas_subterranea)
dfCalidadSubterranea = pd.read_csv(path_calidad_subterranea)

In [24]:
dfEscalasSubterranea.head()

,CALIDAD DEL AGUA PARA COLIFORMES FECALES_subterraneas,CRITERIO,DESCRIPCION
0,Excelente,COLI_FEC menor a 1.1,Agua potable. Agua no contaminada o condicion ...
1,Buena calidad,COLI_FEC mayor o igual de 1.1 y menor o igual ...,Aguas para uso recreativo con contacto primari...
2,Aceptable,COLI_FEC mayor de 200 y menor o igual a 1000,Aguas con calidad admisible como fuente de aba...
3,Contaminada,COLI_FEC mayor de 1000 y menor o igual a 10000,Aguas con contaminacion bacteriologica. Indic...
4,Fuertemente contaminada,COLI_FEC mayor de 10000,Aguas con fuerte contaminacion bacteriologica....


In [25]:
dfEscalasSubterranea.describe()

,CALIDAD DEL AGUA PARA COLIFORMES FECALES_subterraneas,CRITERIO,DESCRIPCION
count,66,66,66
unique,47,52,29
top,Potable - Excelente,CRITERIO,DESCRIPCION
freq,8,14,14


In [26]:
dfCalidadSubterranea.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


In [27]:
dfCalidadSubterranea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CLAVE                 1068 non-null   object 
 1   SITIO                 1068 non-null   object 
 2   ORGANISMO_DE_CUENCA   1068 non-null   object 
 3   ESTADO                1068 non-null   object 
 4   MUNICIPIO             1068 non-null   object 
 5   ACUIFERO              1068 non-null   object 
 6   SUBTIPO               1068 non-null   object 
 7   LONGITUD              1068 non-null   float64
 8   LATITUD               1068 non-null   float64
 9   PERIODO               1068 non-null   int64  
 10  ALC_mg/L              1064 non-null   float64
 11  CALIDAD_ALC           1064 non-null   object 
 12  CONDUCT_mS/cm         1062 non-null   float64
 13  CALIDAD_CONDUC        1062 non-null   object 
 14  SDT_mg/L              0 non-null      float64
 15  SDT_M_mg/L           

In [28]:
dfCalidadSubterranea.isna().sum()

CLAVE                      0
SITIO                      0
ORGANISMO_DE_CUENCA        0
ESTADO                     0
MUNICIPIO                  0
ACUIFERO                   0
SUBTIPO                    0
LONGITUD                   0
LATITUD                    0
PERIODO                    0
ALC_mg/L                   4
CALIDAD_ALC                4
CONDUCT_mS/cm              6
CALIDAD_CONDUC             6
SDT_mg/L                1068
SDT_M_mg/L                 2
CALIDAD_SDT_ra             2
CALIDAD_SDT_salin          2
FLUORUROS_mg/L             0
CALIDAD_FLUO               0
DUR_mg/L                   1
CALIDAD_DUR                1
COLI_FEC_NMP/100_mL        0
CALIDAD_COLI_FEC           0
N_NO3_mg/L                 1
CALIDAD_N_NO3              1
AS_TOT_mg/L                0
CALIDAD_AS                 0
CD_TOT_mg/L                0
CALIDAD_CD                 0
CR_TOT_mg/L                0
CALIDAD_CR                 0
HG_TOT_mg/L                0
CALIDAD_HG                 0
PB_TOT_mg/L   

Borrar columna Clave por que no es necesaria, SDT_mg/L  todo esta vacío.


In [49]:
#del dfCalidadSubterranea['SDT_mg/L']
#del dfCalidadSubterranea['CLAVE']

In [37]:
dfCalidadSubterranea.isna().sum()

SITIO                     0
ORGANISMO_DE_CUENCA       0
ESTADO                    0
MUNICIPIO                 0
ACUIFERO                  0
SUBTIPO                   0
LONGITUD                  0
LATITUD                   0
PERIODO                   0
ALC_mg/L                  4
CALIDAD_ALC               4
CONDUCT_mS/cm             6
CALIDAD_CONDUC            6
SDT_M_mg/L                2
CALIDAD_SDT_ra            2
CALIDAD_SDT_salin         2
FLUORUROS_mg/L            0
CALIDAD_FLUO              0
DUR_mg/L                  1
CALIDAD_DUR               1
COLI_FEC_NMP/100_mL       0
CALIDAD_COLI_FEC          0
N_NO3_mg/L                1
CALIDAD_N_NO3             1
AS_TOT_mg/L               0
CALIDAD_AS                0
CD_TOT_mg/L               0
CALIDAD_CD                0
CR_TOT_mg/L               0
CALIDAD_CR                0
HG_TOT_mg/L               0
CALIDAD_HG                0
PB_TOT_mg/L               0
CALIDAD_PB                0
MN_TOT_mg/L               0
CALIDAD_MN          

In [50]:
###De las columnas que contienen nulos borrar registros con mas de 2 nulos,

col_lst = ['CONTAMINANTES', 'CALIDAD_N_NO3','N_NO3_mg/L','CALIDAD_DUR', 'DUR_mg/L', 'SDT_M_mg/L','CALIDAD_SDT_ra','CALIDAD_SDT_salin', 'ALC_mg/L','CALIDAD_ALC','CONDUCT_mS/cm','CALIDAD_CONDUC']

dfCalidadSubterranea.dropna(axis = 0, subset = col_lst, thresh=2, inplace = True)

In [47]:
dfCalidadSubterranea.isna().sum()

SITIO                     0
ORGANISMO_DE_CUENCA       0
ESTADO                    0
MUNICIPIO                 0
ACUIFERO                  0
SUBTIPO                   0
LONGITUD                  0
LATITUD                   0
PERIODO                   0
ALC_mg/L                  4
CALIDAD_ALC               4
CONDUCT_mS/cm             6
CALIDAD_CONDUC            6
SDT_M_mg/L                2
CALIDAD_SDT_ra            2
CALIDAD_SDT_salin         2
FLUORUROS_mg/L            0
CALIDAD_FLUO              0
DUR_mg/L                  1
CALIDAD_DUR               1
COLI_FEC_NMP/100_mL       0
CALIDAD_COLI_FEC          0
N_NO3_mg/L                1
CALIDAD_N_NO3             1
AS_TOT_mg/L               0
CALIDAD_AS                0
CD_TOT_mg/L               0
CALIDAD_CD                0
CR_TOT_mg/L               0
CALIDAD_CR                0
HG_TOT_mg/L               0
CALIDAD_HG                0
PB_TOT_mg/L               0
CALIDAD_PB                0
MN_TOT_mg/L               0
CALIDAD_MN          

### Ningún registro tiene más de 2 campos nulos.

In [66]:
### Categoricas y numéricas con nulos para imputar.

categoricas = ['CALIDAD_SDT_ra','CALIDAD_SDT_salin','CALIDAD_N_NO3','CALIDAD_DUR','CALIDAD_ALC','CALIDAD_CONDUC']

numericas = [ 'N_NO3_mg/L', 'DUR_mg/L', 'SDT_M_mg/L', 'ALC_mg/L','CONDUCT_mS/cm',]

####'CONTAMINANTES'?????


In [96]:
dfCalidadSubterranea['DUR_mg/L']

0        213.732
1       185.0514
2        120.719
3        199.879
4       476.9872
          ...   
1063     752.096
1064         273
1065    660.2126
1066     406.368
1067     362.544
Name: DUR_mg/L, Length: 1068, dtype: object

In [106]:
###convertimos datos no numericos.

dfCalidadSubterranea.loc[ dfCalidadSubterranea['N_NO3_mg/L'] == '<0.02', 'N_NO3_mg/L'] = 0.02

dfCalidadSubterranea.loc[ dfCalidadSubterranea['DUR_mg/L'] == '<20', 'DUR_mg/L'] = 20

dfCalidadSubterranea.loc[ dfCalidadSubterranea['SDT_M_mg/L'] == '<25', 'SDT_M_mg/L'] = 25

dfCalidadSubterranea['N_NO3_mg/L'] = dfCalidadSubterranea['N_NO3_mg/L'].astype(float)

dfCalidadSubterranea['DUR_mg/L'] = dfCalidadSubterranea['DUR_mg/L'].astype(float)

dfCalidadSubterranea['SDT_M_mg/L'] = dfCalidadSubterranea['SDT_M_mg/L'].astype(float)

In [108]:
dfCalidadSubterranea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 56 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SITIO                 1068 non-null   object 
 1   ORGANISMO_DE_CUENCA   1068 non-null   object 
 2   ESTADO                1068 non-null   object 
 3   MUNICIPIO             1068 non-null   object 
 4   ACUIFERO              1068 non-null   object 
 5   SUBTIPO               1068 non-null   object 
 6   LONGITUD              1068 non-null   float64
 7   LATITUD               1068 non-null   float64
 8   PERIODO               1068 non-null   int64  
 9   ALC_mg/L              1064 non-null   float64
 10  CALIDAD_ALC           1064 non-null   object 
 11  CONDUCT_mS/cm         1062 non-null   float64
 12  CALIDAD_CONDUC        1062 non-null   object 
 13  SDT_M_mg/L            1066 non-null   float64
 14  CALIDAD_SDT_ra        1066 non-null   object 
 15  CALIDAD_SDT_salin    

In [127]:
for key in numericas:
    mean_value=dfCalidadSubterranea[key].mean()
    dfCalidadSubterranea[key].fillna(value=mean_value, inplace=True)

for key in categoricas:
    mode=dfCalidadSubterranea[key].mode()
    print(mode)
    dfCalidadSubterranea[key].fillna(value=mode , inplace=True)

0    Excelente para riego
Name: CALIDAD_SDT_ra, dtype: object
0    Potable - Dulce
Name: CALIDAD_SDT_salin, dtype: object
0    Potable - Excelente
Name: CALIDAD_N_NO3, dtype: object
0    Potable - Dura
Name: CALIDAD_DUR, dtype: object
0    Alta
Name: CALIDAD_ALC, dtype: object
0    Permisible para riego
Name: CALIDAD_CONDUC, dtype: object


In [121]:
dfCalidadSubterranea['CALIDAD_CONDUC']

0       Permisible para riego
1            Buena para riego
2            Buena para riego
3            Buena para riego
4       Permisible para riego
                ...          
1063        Dudosa para riego
1064         Buena para riego
1065        Dudosa para riego
1066    Permisible para riego
1067    Permisible para riego
Name: CALIDAD_CONDUC, Length: 1068, dtype: object

In [72]:
### de una vez normalizamos?

numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='most_frequent'))])

transformer = ColumnTransformer(
    transformers=[
        ('Numericas', numeric_transformer, numericas),
        ('Categoricas', categorical_transformer, categoricas)
    ],
    remainder='passthrough'
)


In [117]:
###Revisamos de nuevo cuantos nulos hay

dfCalidadSubterranea.isna().sum()

SITIO                      0
ORGANISMO_DE_CUENCA        0
ESTADO                     0
MUNICIPIO                  0
ACUIFERO                   0
SUBTIPO                    0
LONGITUD                   0
LATITUD                    0
PERIODO                    0
ALC_mg/L                   0
CALIDAD_ALC                4
CONDUCT_mS/cm              0
CALIDAD_CONDUC             6
SDT_M_mg/L                 0
CALIDAD_SDT_ra             2
CALIDAD_SDT_salin          2
FLUORUROS_mg/L             0
CALIDAD_FLUO               0
DUR_mg/L                   0
CALIDAD_DUR                1
COLI_FEC_NMP/100_mL        0
CALIDAD_COLI_FEC           0
N_NO3_mg/L                 0
CALIDAD_N_NO3              1
AS_TOT_mg/L                0
CALIDAD_AS                 0
CD_TOT_mg/L                0
CALIDAD_CD                 0
CR_TOT_mg/L                0
CALIDAD_CR                 0
HG_TOT_mg/L                0
CALIDAD_HG                 0
PB_TOT_mg/L                0
CALIDAD_PB                 0
MN_TOT_mg/L   